In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
import time
import re
import pandas as pd
import pickle
import os

In [82]:
chrome_options = Options()
timeout = 60
# chrome_options.add_argument('--no-sandbox')
# browser = webdriver.Chrome('chromedriver') #, options=chrome_options
browser = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/home/kirill/.wdm/drivers/chromedriver/linux64/103.0.5060.53/chromedriver] found in cache


In [83]:
url = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_entry_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222021-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222021-12-31T00:00:00%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_entry_date%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_common_parts_law_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D1%81%D1%82.%2020.2%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%221f074cdd-f56a-4089-b901-df7a05b3a5c8%22,%22operator%22:%22AND%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22asc%22%7D%5D,%22simpleSearchFieldsBundle%22:null,%22groups%22:%5B%22%D0%94%D0%B5%D0%BB%D0%B0%20%D0%BE%D0%B1%20%D0%90%D0%9F%22%5D,%22noOrpho%22:false,%22start%22:100%7D'
browser.get(url)

In [3]:
url_20_2 = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_entry_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222021-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222021-12-31T00:00:00%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_entry_date%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_common_parts_law_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D1%81%D1%82.%2020.2%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%221f074cdd-f56a-4089-b901-df7a05b3a5c8%22,%22operator%22:%22AND%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22asc%22%7D%5D,%22simpleSearchFieldsBundle%22:null,%22groups%22:%5B%22%D0%94%D0%B5%D0%BB%D0%B0%20%D0%BE%D0%B1%20%D0%90%D0%9F%22%5D,%22noOrpho%22:false,%22start%22:20%7D'

In [29]:
browser.get(url)

In [42]:
last_page = browser.find_elements_by_tag_name('td.pageNumTD')[-1].text
last_page = int(''.join(i for i in last_page if i != ' '))

In [159]:
# switch to table view
WebDriverWait(browser, timeout).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/span[2]/a[2]'))).click()
time.sleep(1)

In [84]:
table = browser.find_element_by_class_name('resultsList').find_element(By.TAG_NAME, "tbody") 

In [33]:
case_user_doc_number = [i for i in table.find_elements_by_xpath('//div[@data-field="case_user_doc_number"]')]
adm_case_user_name = [i.text for i in table.find_elements_by_xpath('//div[@data-field="adm_case_user_name"]')]
article = [i.text for i in table.find_elements_by_xpath('//div[@data-field="case_common_parts_law_article"]')]
entry_date = [i.text for i in table.find_elements_by_xpath('//div[@data-field="case_user_entry_date"]')]
result_date = [i.text for i in table.find_elements_by_xpath('//div[@data-field="case_user_doc_result_date"]')]
subject = [i.text for i in table.find_elements_by_xpath('//div[@data-field="case_doc_subject_rf"]')]
court = [i.text for i in table.find_elements_by_xpath('//div[@data-field="case_user_doc_court"]')]
user_name = [i.text for i in table.find_elements_by_xpath('//div[@data-field="adm_case_user_name"]')]
result = [i.text for i in table.find_elements_by_xpath('//div[@data-field="case_user_doc_result"]')]

In [86]:
if table.find_elements_by_xpath('//div[@data-field="case_user_docsff_number"]'):
    print(True)
    

<bound method WebElement.find_elements_by_xpath of <selenium.webdriver.remote.webelement.WebElement (session="82b5c3450e43442607f50ee86b7cf8bd", element="9708c83d-1f5e-4481-86fa-ba001ae0883c")>>

In [54]:
from collections import defaultdict

elem_for_search = [
    "case_user_doc_number",
    "adm_case_user_name",
    "case_common_parts_law_article",
    "case_user_entry_date",
    "case_doc_subject_rf",
    "case_user_doc_court",
    "case_user_doc_result",
    "case_user_doc_result_date", 
    'link'
]

data = defaultdict(list,{ k:[] for k in elem_for_search })

def scrape_elements(data,  table, elem_for_search):
    row_count = len(table.find_elements_by_tag_name('tr'))
    for el in elem_for_search:
        if el == 'link':
            data[el].append([i.get_attribute('href') for i in table.find_elements_by_xpath('//a[@class="cardAction openCardLink"]')])
            
        else:
            search = f'//div[@data-field="{el}"]'
            if table.find_elements_by_xpath:
                data[el].append([i.text for i in table.find_elements_by_xpath(search)])
            else:
                data[el].append([None for i in range(row_count)])

In [55]:
scrape_elements(data,  table, elem_for_search)

In [57]:
data['link']

[['https://bsr.sudrf.ru/bigs/showDocument.html#id=f80c9687111fa7d9b31e06c87e2480a0&shard=%D0%94%D0%B5%D0%BB%D0%B0%20%D0%BE%D0%B1%20%D0%90%D0%9F&from=p&r={%22start%22:100,%22rows%22:10,%22uid%22:%2232347e59-ad9c-43f9-b737-ecf6435cb500%22,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:{%22queryRequests%22:[{%22type%22:%22Q%22,%22request%22:%22{\\%22mode\\%22:\\%22EXTENDED\\%22,\\%22typeRequests\\%22:[{\\%22fieldRequests\\%22:[{\\%22name\\%22:\\%22case_user_doc_entry_date\\%22,\\%22operator\\%22:\\%22B\\%22,\\%22query\\%22:\\%222021-01-01T00:00:00\\%22,\\%22sQuery\\%22:\\%222021-12-31T00:00:00\\%22,\\%22fieldName\\%22:\\%22case_user_doc_entry_date\\%22},{\\%22name\\%22:\\%22case_common_parts_law_article\\%22,\\%22operator\\%22:\\%22EX\\%22,\\%22query\\%22:\\%22%D1%81%D1%82.%2020.2\\%22,\\%22sQuery\\%22:null}],\\%22mode\\%22:\\%22AND\\%22,\\%22name\\%22:\\%22common\\%22,\\%22typesMode\\%22:\\%22AND\\%22}]}%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22},{%22type%2

10

In [9]:
article_19_3_2021 = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_entry_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222021-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222021-12-31T00:00:00%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_entry_date%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_common_parts_law_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%2219.3%5C%22,%5C%22fieldName%5C%22:%5C%22case_common_parts_law_article_cat%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_doc_instance%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D0%9F%D0%B5%D1%80%D0%B2%D0%B0%D1%8F%20%D0%B8%D0%BD%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D1%8F%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22case_common_parts_law_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D1%81%D1%82.%2019.3%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%221f074cdd-f56a-4089-b901-df7a05b3a5c8%22,%22operator%22:%22AND%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:null,%22groups%22:%5B%22%D0%94%D0%B5%D0%BB%D0%B0%20%D0%BE%D0%B1%20%D0%90%D0%9F%22%5D,%22noOrpho%22:false%7D'

In [19]:
article_214_2021 = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20214%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22%D0%A3%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%B5%D0%BB%D0%B0%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D,%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_entry_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222021-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222021-12-31T00:00:00%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_entry_date%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:null,%22groups%22:%5B%22%D0%A3%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%B5%D0%BB%D0%B0%22%5D,%22noOrpho%22:false%7D'

In [20]:
article_214_2022 = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20214%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22%D0%A3%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%B5%D0%BB%D0%B0%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D,%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_entry_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222022-01-01T00:00:00%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_entry_date%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:null,%22groups%22:%5B%22%D0%A3%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%B5%D0%BB%D0%B0%22%5D,%22noOrpho%22:false,%22start%22:0%7D'

### full script

In [23]:
def to_pickle(data, file_name):
    with open(file_name, "wb") as fp:
        pickle.dump(data, fp)

In [58]:
page = 0
url = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_entry_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222021-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222021-12-31T00:00:00%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_entry_date%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_common_parts_law_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22%D1%81%D1%82.%2020.2%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%221f074cdd-f56a-4089-b901-df7a05b3a5c8%22,%22operator%22:%22AND%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22asc%22%7D%5D,%22simpleSearchFieldsBundle%22:null,%22groups%22:%5B%22%D0%94%D0%B5%D0%BB%D0%B0%20%D0%BE%D0%B1%20%D0%90%D0%9F%22%5D,%22noOrpho%22:false'
browser = webdriver.Chrome(ChromeDriverManager().install())

failed_pages = []

for page in range(page, 110, 10):
    print(f'page {page}')
    page_url = url + f',%22start%22:{page}%7D'
    browser.get(page_url)
    time.sleep(5)
    
    if page == 0:
        try:
            WebDriverWait(browser, timeout).until(
                lambda browser: browser.find_elements(By.XPATH, "//div[@class='bgs-result']"))
            print('page loaded')
            time.sleep(1)
            WebDriverWait(browser, timeout).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/span[2]/a[2]'))).click()
        except:
            print('error')
            failed_pages.append(page)
    
    if page % 5 == 0:
        to_pickle(data, '20_2_data')
        print('Progress saved')
    
    try:
        WebDriverWait(browser, timeout).until(
                lambda browser: browser.find_elements(By.XPATH, "//ul[@class='resultsList']"))
        table = browser.find_element_by_class_name('resultsList').find_element(By.TAG_NAME, "tbody") 
        scrape_elements(data, table, elem_for_search)
        
    except TimeoutException:
        print('time out')
        failed_pages.append(page)
    except:
        failed_pages.append(page)
        
to_pickle(data, '20_2_data')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/home/kirill/.wdm/drivers/chromedriver/linux64/103.0.5060.53/chromedriver] found in cache


page 0
page loaded
Progress saved
page 10
Progress saved
page 20
Progress saved
page 30
Progress saved
page 40
Progress saved
page 50
Progress saved
page 60
Progress saved
page 70
Progress saved
page 80
Progress saved
page 90
Progress saved
page 100
Progress saved


In [25]:
with open("data_214_1010", "rb") as fp:
    data_1 = pickle.load(fp)

In [26]:
for k, i in data_1.items():
    print(len(i), k)

17 case_user_doc_number
17 adm_case_user_name
17 case_common_parts_law_article
17 case_user_entry_date
17 case_doc_subject_rf
17 case_user_doc_court
17 case_user_doc_result
17 case_user_doc_result_date
17 link


In [32]:
df = None
for f in os.listdir('data/214_2022'):
    path = 'data/214_2022/' + f
    with open(path, 'rb') as fp:
        data = pickle.load(fp)
        
    for k in data.keys():
        data[k] = [c for i in data[k] for c in i]
        
    if df is None:
        df = pd.DataFrame(data)
    else:
        df = pd.concat([pd.DataFrame(data), df])

In [33]:
df['year'] = 2022

In [34]:
df = df.reset_index()

In [35]:
df.drop('index', axis=1, inplace=True)

In [36]:
df.to_csv('214_year_2022.csv')

In [37]:
df

,case_user_doc_number,adm_case_user_name,case_common_parts_law_article,case_user_entry_date,case_doc_subject_rf,case_user_doc_court,case_user_doc_result,case_user_doc_result_date,link,year
0,22К-2134/2022,None,None,25.03.2022,Краснодарский край,Краснодарский краевой суд,ВЫНЕСЕНО РЕШЕНИЕ (ОПРЕДЕЛЕНИЕ),13.04.2022,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
1,7У-2622/2022,None,None,24.02.2022,Кемеровская область - Кузбасс,Восьмой кассационный суд общей юрисдикции,,,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
2,22-1968/2022,None,None,14.03.2022,Ростовская область,Ростовский областной суд,ВЫНЕСЕНО РЕШЕНИЕ (ОПРЕДЕЛЕНИЕ),,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
3,22К-2927/2022,None,None,04.04.2022,город Санкт-Петербург,Санкт-Петербургский городской суд,ВЫНЕСЕНО РЕШЕНИЕ (ОПРЕДЕЛЕНИЕ),,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
4,22К-1070/2022,None,None,18.02.2022,Приморский край,Приморский краевой суд,ВЫНЕСЕНО РЕШЕНИЕ (ОПРЕДЕЛЕНИЕ),09.03.2022,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
...,...,...,...,...,...,...,...,...,...,...
112,1-26/2022,None,None,27.06.2022,Орловская область,Колпнянский районный суд,,,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
113,1-393/2022,None,None,25.04.2022,Воронежская область,Левобережный районный суд г. Воронежа,,,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
114,2-24/2022,None,None,23.05.2022,Ростовская область,Ростовский областной суд,,,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022
115,1-93/2022,None,None,14.04.2022,Республика Коми,Сыктывдинский районный суд,Вынесен ПРИГОВОР,,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2022


In [17]:
df

,case_user_doc_number,adm_case_user_name,case_common_parts_law_article,case_user_entry_date,case_doc_subject_rf,case_user_doc_court,case_user_doc_result,case_user_doc_result_date,link,year
0,5-4017/2021,Такавдинов Мирзабек Магомедович,ст.19.3 ч.1 КоАП РФ,16.07.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,16.07.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
1,5-4022/2021,Ибрагимов Альберт Абдуллахович,ст.19.3 ч.1 КоАП РФ,31.07.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,31.07.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
2,5-4075/2021,Алиев Адам Сидикович,ст.19.3 ч.1 КоАП РФ,09.08.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,09.08.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
3,5-4018/2021,Магомедов Саид Шамилович,ст.19.3 ч.1 КоАП РФ,26.07.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,26.07.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
4,5-651/2021,Умаров Микаил Муратович,ст.19.3 ч.1 КоАП РФ,05.02.2021,Республика Дагестан,Кумторкалинский районный суд,Вынесено постановление о назначении администра...,05.02.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
...,...,...,...,...,...,...,...,...,...,...
57846,5-978/2021,Баушениек Денис Валерьевич,ст.19.3 ч.1 КоАП РФ,26.10.2021,Ростовская область,Мясниковский районный суд,Вынесено постановление о назначении администра...,10.12.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
57847,5-1050/2021,Чучиян Дмитрий Иличевич,ст.19.3 ч.1 КоАП РФ,17.11.2021,Ростовская область,Мясниковский районный суд,Вынесено постановление о назначении администра...,17.11.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
57848,5-878/2021,Телесов Арчын Олегович,ст.19.3 ч.1 КоАП РФ,24.11.2021,Республика Алтай,Улаганский районный суд,Вынесено постановление о назначении администра...,24.11.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
57849,5-782/2021,Саналов Сергей Валерьевич,ст.19.3 ч.1 КоАП РФ,09.10.2021,Республика Алтай,Улаганский районный суд,Вынесено постановление о назначении администра...,09.10.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021


In [4]:
df = pd.read_csv('19_3_year_2021.csv', index_col=0)

In [8]:
df.columns

Index(['case_user_doc_number', 'adm_case_user_name',
       'case_common_parts_law_article', 'case_user_entry_date',
       'case_doc_subject_rf', 'case_user_doc_court', 'case_user_doc_result',
       'case_user_doc_result_date', 'link', 'year'],
      dtype='object')

In [7]:
df.head()

,case_user_doc_number,adm_case_user_name,case_common_parts_law_article,case_user_entry_date,case_doc_subject_rf,case_user_doc_court,case_user_doc_result,case_user_doc_result_date,link,year
0,5-4017/2021,Такавдинов Мирзабек Магомедович,ст.19.3 ч.1 КоАП РФ,16.07.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,16.07.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
1,5-4022/2021,Ибрагимов Альберт Абдуллахович,ст.19.3 ч.1 КоАП РФ,31.07.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,31.07.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
2,5-4075/2021,Алиев Адам Сидикович,ст.19.3 ч.1 КоАП РФ,09.08.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,09.08.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
3,5-4018/2021,Магомедов Саид Шамилович,ст.19.3 ч.1 КоАП РФ,26.07.2021,Республика Дагестан,Кизилюртовский городской суд,Вынесено постановление о назначении администра...,26.07.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
4,5-651/2021,Умаров Микаил Муратович,ст.19.3 ч.1 КоАП РФ,05.02.2021,Республика Дагестан,Кумторкалинский районный суд,Вынесено постановление о назначении администра...,05.02.2021,https://bsr.sudrf.ru/bigs/showDocument.html#id...,2021
